In [ ]:
import accelerate
accelerate.__version__

In [ ]:
!pip install -U accelerate==0.18.0

In [ ]:
!pip install evaluate

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import sys

from sklearn.model_selection import StratifiedKFold

import timm
from timm.models import create_model,build_model_with_cfg
from timm.optim import create_optimizer_v2
from timm.scheduler import create_scheduler_v2
from timm.loss import LabelSmoothingCrossEntropy,BinaryCrossEntropy,JsdCrossEntropy
from timm.utils import AverageMeter,ModelEmaV2

import os
import numpy as np
import pandas as pd
from PIL import Image

from accelerate import Accelerator
from accelerate.utils import set_seed

from tqdm import tqdm
from IPython.display import clear_output

In [ ]:
import evaluate

In [ ]:
import wandb
wandb.login(key="")

In [ ]:
timm.list_models('*convn*',pretrained=True)

In [ ]:
ckp = 'convnext_large_mlp.clip_laion2b_soup_ft_in12k_in1k_384'
model = create_model(ckp,in_chans=3, num_classes=2, pretrained=True)
data_config = timm.data.resolve_model_data_config(model)
transform = timm.data.create_transform(**data_config, is_training=False)

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open("/kaggle/input/hackathon-online-find-the-buildings/train/train/"+row["file"])
        image = image.convert("RGB")
        image = self.transform(image)
        label = row['label']
        return image, label

In [ ]:
def get_dataloaders(datasets,batch_size: int = 64):
    train_size = int(0.9 * len(datasets))
    test_size = len(datasets) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(
        datasets, [train_size, test_size])

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                                    pin_memory=True,
                                                    shuffle=True,
                                                    num_workers=2)
    eval_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size*2,
                                                    pin_memory=True,
                                                    shuffle=False,
                                                    num_workers=2)
    return train_dataloader, eval_dataloader

In [ ]:
df = pd.read_csv("/kaggle/input/hackathon-online-find-the-buildings/train.csv")

In [ ]:
def training_function(model,df):
    set_seed(43)
    num_epochs=100
    lr=(5e-6)
    batch_size = 8
    mixed_precision="fp16"
    accelerator = Accelerator(mixed_precision=mixed_precision,
                              gradient_accumulation_steps=1,
                              log_with="wandb")
    
    optimizer = create_optimizer_v2(
        model, opt="adamw", lr=lr, weight_decay=(1e-2))
    scheduler = timm.scheduler.CosineLRScheduler(optimizer, t_initial=2)
    criterion = LabelSmoothingCrossEntropy(smoothing=0.1)
    accuracy = evaluate.load("accuracy")
    
    dataset = Dataset(df,transform)
    train_dataloader, eval_dataloader = get_dataloaders(dataset,batch_size)
    
    model, optimizer, train_dataloader, eval_dataloader,scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader,scheduler)
    best_score = -1.0
    accelerator.init_trackers("binary classification")
    
    for epoch in range(num_epochs):
        model.train()
        
        num_steps_per_epoch = len(train_dataloader)
        num_updates = epoch * num_steps_per_epoch
        
        train_meters = {
            'loss': AverageMeter(),
            'acc': AverageMeter(),
        }
        
        if accelerator.is_local_main_process:
            train_bar = tqdm(total=len(train_dataloader))

        for batch in train_dataloader:
            clear_output()
            with accelerator.accumulate(model):
                x, y = batch
                out = model(x)
                with accelerator.autocast():
                    loss = criterion(out, y)
                _,pred = out.max(1)

                accelerator.backward(loss)
                optimizer.step()
                optimizer.zero_grad()

                scheduler.step_update(num_updates=num_updates)

            predictions, references = accelerator.gather_for_metrics((pred,y))

            acc = accuracy.compute(predictions=predictions, references=references)["accuracy"]

            train_meters['loss'].update(loss, n=x.size(0))
            train_meters['acc'].update(acc, n=x.size(0))
                
                
            if accelerator.is_local_main_process:
                train_bar.update(1)
                train_bar.set_description(
                        f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item():.4f} , Acc: {acc:.4f}")
                    
        scheduler.step(epoch + 1)

        loss_avg = train_meters['loss'].avg
        acc_avg = train_meters['acc'].avg
        accelerator.log({"loss":loss_avg})
        accelerator.log({"acc":acc_avg})
        if accelerator.is_local_main_process:
                train_bar.set_description(
                        f"Epoch {epoch+1}/{num_epochs}, Training Loss Avg: {loss_avg:.4f} , Acc Avg: {acc_avg:.4f}")
        model.eval()
        val_meters = {
            'loss': AverageMeter(),
            'acc': AverageMeter(),
        }
        if accelerator.is_local_main_process:
            val_bar = tqdm(total=len(eval_dataloader))

        with torch.no_grad():
            for batch in eval_dataloader:
                clear_output()
                x, y = batch
                out = model(x)
                with accelerator.autocast():
                    loss = criterion(out, y)
                val_loss = loss.item()
                _,pred = out.max(1)

                
                predictions, references = accelerator.gather_for_metrics((pred,y))
                val_acc = accuracy.compute(predictions=predictions, references=references)["accuracy"]

                val_meters['loss'].update(val_loss , n=x.size(0))
                val_meters['acc'].update(val_acc, n=x.size(0))
                if accelerator.is_local_main_process:
                    val_bar.update(1)
                    val_bar.set_description(
                            f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {loss.item():.4f} , Acc: {val_acc:.4f}")

        loss_avg = val_meters['loss'].avg
        acc_avg = val_meters['acc'].avg
        accelerator.log({"val_loss":loss_avg})
        accelerator.log({"val_acc":acc_avg})
        if accelerator.is_local_main_process:
            val_bar.set_description(
                f"Epoch {epoch+1}/{num_epochs}, Validation Loss Avg: {loss_avg:.4f} , Acc Avg: {acc_avg:.4f}")
            
    
        if val_meters['acc'].avg > best_score:
                best_score = val_meters['acc'].avg
                accelerator.wait_for_everyone()
                accelerator.save(accelerator.get_state_dict(model),f"./{ckp}.pth")
                accelerator.print(f'Save best model epoch:' + str(epoch + 1))
        accelerator.free_memory()
    accelerator.end_training()

In [ ]:
from accelerate import notebook_launcher
notebook_launcher(training_function,(model,df,),num_processes=2)

In [ ]:
model = create_model("convnext_large_mlp.clip_laion2b_soup_ft_in12k_in1k_384",
                        in_chans=3, num_classes=2,checkpoint_path="/kaggle/working/convnext_large_mlp.clip_laion2b_soup_ft_in12k_in1k_384.pth").to("cuda")
data_config = timm.data.resolve_model_data_config(model)
transform = timm.data.create_transform(**data_config, is_training=False)

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open("/kaggle/input/hackathon-online-find-the-buildings/test/"+row['file'])
        image = image.convert("RGB")
        image = self.transform(image)
        return image

In [ ]:
batch_size = 100
testDf = pd.read_csv("/kaggle/input/hackathon-online-find-the-buildings/test.csv")
dataset = TestDataset(testDf,transform)
dataloader = torch.utils.data.DataLoader(dataset, pin_memory=True,
                                                  num_workers=2,batch_size=batch_size)

In [ ]:
import torch.nn.functional as nnf
preds = np.array([])
probs = np.array([])
for batch in tqdm(dataloader):
    model.eval()
    with torch.no_grad():
        output = model(batch.to("cuda"))
    prob = nnf.softmax(output, dim=1)
    top_p, top_class = prob.topk(1, dim = 1)
    preds = np.append(preds, top_class.cpu().numpy())
    probs = np.append(probs, top_p.cpu().numpy())

In [ ]:
testDf["label"] = preds.astype("int")
#testDf["probs"] = probs

testDf["label"].value_counts()

In [ ]:
testDf.to_csv("submit.csv",index=0)